In [7]:
import torch
import torch.nn as nn
import torch.onnx

# Define model

In [8]:
# Define a simple neural network model
class SimpleModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SimpleModel, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.gelu = nn.GELU()
        self.fc2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = self.fc1(x)
        x = self.gelu(x)
        x = self.fc2(x)
        return x

In [9]:
# Define the input size, hidden layer size, and output size
input_size = 10
hidden_size = 5
output_size = 1

In [10]:
# Instantiate the model
model = SimpleModel(input_size, hidden_size, output_size)

# Test model

In [14]:
# Define a sample input tensor to check the model
sample_input = torch.randn((3, input_size))  # Batch size of 3
output = model(sample_input)
"input", sample_input, "output", output

('input',
 tensor([[-0.1674,  2.6486,  0.8259,  0.7422,  3.4326, -0.8476,  1.4679, -1.6526,
          -0.6042,  1.1872],
         [-1.7339, -0.3402,  0.1187,  0.2542,  0.1221,  0.6766,  0.2905,  0.5238,
           0.4045,  0.3253],
         [-2.5379,  1.3199, -0.8511,  0.6519,  1.9053,  0.8705,  0.3307, -0.0951,
           0.7305,  0.5680]]),
 'output',
 tensor([[ 0.1585],
         [-0.0206],
         [ 0.0089]], grad_fn=<AddmmBackward0>))

# Export to ONNX

In [15]:
model.eval()

dummy_input = torch.randn(1, input_size)

onnx_path = "simple_model.onnx"

In [18]:
torch.onnx.export(
    model,
    dummy_input,
    onnx_path,
    input_names=["input"],
    output_names=["output"],
    verbose=True
)

Exported graph: graph(%input : Float(1, 10, strides=[10, 1], requires_grad=0, device=cpu),
      %fc1.weight : Float(5, 10, strides=[10, 1], requires_grad=1, device=cpu),
      %fc1.bias : Float(5, strides=[1], requires_grad=1, device=cpu),
      %fc2.weight : Float(1, 5, strides=[5, 1], requires_grad=1, device=cpu),
      %fc2.bias : Float(1, strides=[1], requires_grad=1, device=cpu)):
  %/fc1/Gemm_output_0 : Float(1, 5, strides=[5, 1], requires_grad=1, device=cpu) = onnx::Gemm[alpha=1., beta=1., transB=1, onnx_name="/fc1/Gemm"](%input, %fc1.weight, %fc1.bias), scope: __main__.SimpleModel::/torch.nn.modules.linear.Linear::fc1 # /home/jakub/PycharmProjects/godot-onnx-demo/python/.venv/lib/python3.11/site-packages/torch/nn/modules/linear.py:114:0
  %/gelu/Constant_output_0 : Float(requires_grad=0, device=cpu) = onnx::Constant[value={1.41421}, onnx_name="/gelu/Constant"](), scope: __main__.SimpleModel::/torch.nn.modules.activation.GELU::gelu # /home/jakub/PycharmProjects/godot-onnx-demo/

In [21]:
export_output = torch.onnx.dynamo_export(
    model,
    dummy_input
)

export_output.save("simple_model_dynamo.onnx")

/home/jakub/PycharmProjects/godot-onnx-demo/python/.venv/lib/python3.11/site-packages/torch/onnx/_internal/exporter.py:130: UserWarning: torch.onnx.dynamo_export only implements opset version 18 for now. If you need to use a different opset version, please register them with register_custom_op.
  warnings.warn(
